# Running BerkeleyGW with BGWpy #

In this notebook, we assume that you are somewhat familiar with the BerkeleyGW software: what problem it solves, and what is the general workflow to run it. We also assume that you have a basic knowledge of Python and its terminology.


Before you begin, make sure that you have the following packages installed:

* Jupyter Notebook
* QuantumEspresso
* BerkeleyGW
* BGWpy

To run BGWpy, you'll also need the `bin` directories of BerkeleyGW and Abinit installations located in your `PATH` environment variable.

## Checking your configuration ##

The following cell is used to generate information that we'll need, should we have to debug this notebook.  You don't need to run it, but it may be useful to look at for educational purposes.

In [ ]:
import sys
import os
import BGWpy.config as defaults

print("Python kernel:\n    {}                     ".format(sys.executable))
print("Python version:\n    {}                    ".format(sys.version))
print("Current working directory:\n    {}         ".format(os.getcwd()))
print("Configuration file:\n     {}               ".format(defaults.config_file))
print("Use HDF5?:\n    {}                         ".format(defaults.flavors['use_hdf5']))
print("Use complex version of BerkeleyGW?:\n    {}".format(defaults.flavors['flavor_complex']))
print("DFT Flavor:\n    {}                        ".format(defaults.flavors['dft_flavor']))
print("Default MPI settings:\n    {}              ".format(defaults.default_mpi))
print("Default runscript settings:\n    {}        ".format(defaults.default_runscript))
print("Paths in $PATH:")
for i in os.environ['PATH'].split(":"):
    print("    {}".format(i))

Pay attention to the `use_hdf5` flag. It should reflect whether you compiled BerkeleyGW with HDF5 support or not. If the information above is not consistent with what you have, then you should edit your `~/.BGWpyrc` file accordingly. This is important because the file names that BGW expects from a calculation depends on it. If you don't have HDF5, then you should remove all the '.h5' extensions from file names. It is highly recommended, however, that you build BGW with HDF5 support, as it could become mandatory in the future.

If you don't have a `~/.BGWpyrc` yet, you can copy it from the `BGWpy/config` directory, or simply run the script `BGWpy_make_config_file.py`.

# Load Libraries #

First, we load two external packages which BGWpy uses:  `numpy` and `pymatgen`.

In [ ]:
import pymatgen
import numpy as np

Next, we load the `Structure` class from the BGWpy package.  This module is responsible for creating geometries that BGWpy can read and manipulate using the `pymatgen` package.

In [ ]:
from BGWpy import Structure

Next, we load the classes which create and run Quantum ESPRESSO calculations.

In [ ]:
from BGWpy import QeScfTask, QeBgwFlow

Finally, we load the classes with create and run BerkeleyGW calculations.

In [ ]:
from BGWpy import EpsilonTask, SigmaTask, KernelTask, AbsorptionTask

Make sure that both the BerkeleyGW and Quantum ESPRESSO binary folders are in the PATH folder

# Create the Structure #

For this tutorial, we'll calculate the many-body properties of the GaAs primitive cell.  All files that you will need have been provided for you in the `Data` subdirectory.

SHOW PICTURE HERE.  (Even better if can play using `pymatgen`...)

Geometries are specified in BGWpy using pymatgen's `Structure` class, which may be imported directly from BGWpy or through pymatgen.

There are a number of ways that we can import geometries into BGWpy using the `Structure` class.  For example, we can load them from a pre-existing CIF file:

In [ ]:
structure = Structure.from_file('../Data/Structures/GaAs.cif')
print(structure)

We can also load them from a previous pymatgen Structure which has been exported to a file in the JSON format:

In [ ]:
Structure.from_file('../Data/Structures/GaAs.json')
print(structure)

We can even use pymatgen to directly create the structure in a Python script:

In [ ]:
acell_angstrom =  5.6535
rprim = np.array([[.0,.5,.5],[.5,.0,.5],[.5,.5,.0]]) * acell_angstrom
structure = pymatgen.Structure(
    lattice = pymatgen.core.lattice.Lattice(rprim),
    species= ['Ga', 'As'],
    coords = [3*[.0], 3*[.25]],
    )
print(structure)

For more information about pymatgen, please consult its official documentation.

# Generating the Ground State Density #

To begin, we will run a ground state DFT calculation to self-consistency to generate the ground state charge density for the calculation.  This ground state charge density will be fed into all wavefunction calculations in the next step.  We use Quantum ESPRESSO in this notebook, however BerkeleyGW and BGWpy supports a number of other DFT packages.

First, we will create a object of the `QeScfTask` task to prepare the needed variables:

In [ ]:
scf_task = QeScfTask(
    dirname='Runs/11-Density',

    structure=structure,
    prefix='GaAs',
    pseudo_dir='../Data/Pseudos',
    pseudos=['31-Ga.PBE.UPF', '33-As.PBE.UPF'],

    ngkpt=[2, 2, 2],      # k-points grid
    kshift=[.5, .5, .5],  # k-points shift
    ecutwfc=10.0,       # Wavefunctions cutoff energy

    # These are the default parameters for the MPI runner.
    # Please adapt them to your needs.
    nproc=1,
    nproc_per_node=1,
    mpirun='mpirun',
    nproc_flag='-n',
    nproc_per_node_flag='--npernode',
)

As you can see, BGWpy has a number of parameters that you will need to set.  However, many of these parameters are consistent from calculation to calculation, so we'll store them in dictionaries that we can reuse for future steps.

First, a dictionary to store all variables that will be used across all Quantum ESPRESSO calculations:

In [ ]:
qe_general_settings = {
    'prefix': 'GaAs',
    'pseudo_dir': '../Data/Pseudos',
    'pseudos': ['31-Ga.PBE.UPF', '33-As.PBE.UPF'],
}

Next, a dictionary to store the variables which are used only for this particular SCF task:

In [ ]:
qe_scf_settings = {
    'ngkpt': [2, 2, 2],
    'kshift': [.5, .5, .5],
    'ecutwfc': 10.0,
}

And finally, a dictionary to store the settings related to running calculations with MPI.

In [ ]:
mpi_settings = {
    'nproc': 4,
    'mpirun': 'mpirun',
    'nproc_flag': '-n',
    'nproc_per_node_flag': " ", # Set to a single space to omit
    'nproc_per_node': " ", # Set to a single space to omit
}

Note that all these dictionaries correspond to arguments for the `QeScfTask`, stored as key/value pairs.  This allows us to use dictionary unpacking (see TODO for more information) to considerably tidy up our code:

In [ ]:
scf_task = QeScfTask(
    dirname='Runs/11-Density',
    structure=structure,
    **qe_general_settings,
    **qe_scf_settings,
    **mpi_settings,
)

Now that we've created the `QeScfTask` task, we can use the `write` method to write the needed input files to disk:

In [ ]:
scf_task.write()

If you receive an error message stating that an executable could not be found, you likely do not have the needed BerkeleyGW and Quantum ESPRESSO `bin` folders in your `PATH` environment variable.

Let's take a look at the folder that was created by this task using Jupyter's built-in `!ls` magic command:

In [ ]:
!ls 'Runs/11-Density'

In our new folder, there are two input files:

* `scf.in`, the input file for Quantum ESPRESSO, and
* `run.sh`, the script that will be used to run Quantum ESPRESSO.

`GaAs.save` is a folder used by Quantum ESPRESSO to store intermediate files.

Now that we've created the needed input files, let's run the `run.sh` script using the `run` method.  Note that this step will take a few seconds, as it will run Quantum ESPRESSO in the background.

In [ ]:
scf_task.run()

Finally, we can check the status of the calculation using the `report` method.  You should see a message telling you that it's been completed.

In [ ]:
scf_task.report()

Our calculation creates two files that we'll need for the generation of wavefunction files.  We'll store their filenames to reuse later.

In [ ]:
charge_density_fname = scf_task.charge_density_fname
data_file_fname = scf_task.data_file_fname
print("Charge density file name: {}".format(charge_density_fname))
print("Data file name:           {}".format(data_file_fname))

# Generating the Wavefunctions #

Now that we've generated the ground state density, we'll used this to generate the wavefunctions that we'll feed into BerkeleyGW.  This may be done with the ` QeBgwFlow` class.  As mentioned in the introduction, we'll need up to 6 different types of wavefunction files.

## WFN ##

`WFN` is the "standard" k-shifted wavefunction file which is read by the `Epsilon` calculation, and thus is needed for all BerkeleyGW calculations.

It (and all other wavefunction files) are generated using the `QeBgwFlow` class.  The only difference between these wavefunction types are the parameter values used:

In [ ]:
wfn_flow = QeBgwFlow(
    dirname='Runs/12-Wfn',
    structure=structure,

    prefix='GaAs',
    pseudo_dir='Data/Pseudos',
    pseudos=['31-Ga.PBE.UPF', '33-As.PBE.UPF'],

    charge_density_fname='Runs_qe/11-Density/GaAs.save/charge-density.dat',
    data_file_fname='Runs_qe/11-Density/GaAs.save/data-file.xml',

    ngkpt=[2, 2, 2],      # k-points grid
    kshift=[.5, .5, .5],  # k-points shift
    ecutwfc=10.0,       # Wavefunctions cutoff energy
    nbnd=9,             # Number of bands

    # These are the default parameters for the MPI runner.
    # Please adapt them to your needs.
    nproc=1,
    nproc_per_node=1,
    mpirun='mpirun',
    nproc_flag='-n',
    nproc_per_node_flag='--npernode'
)

As before, we will break up these arguments into sets of dictionaries: the settings common to all wavefunction calculations

In [ ]:
qe_wfn_input_files = {
    'charge_density_fname': charge_density_fname,
    'data_file_fname': data_file_fname
}

and the arguments specific to the current wavefunction calculation

In [ ]:
wfn_settings = {
    'ecutwfc': 10.0,
    'ngkpt': [2, 2, 2],
    'kshift': [.5, .5, .5],
    'nbnd': 9,
}

Note that, because we've already set up the `qe_general_settings` and `mpi_settings` dictionaries, we don't need to re-declare them here.

We can now create the instance of the `QeBgwFlow` class:

In [ ]:
wfn_flow = QeBgwFlow(
    dirname='Runs/12-Wfn',
    structure=structure,
    **qe_general_settings,
    **qe_wfn_input_files,
    **wfn_settings,
    **mpi_settings,
)

As before, we'll write the input files to disc then run the calculation:

In [ ]:
wfn_flow.write()
wfn_flow.run()
wfn_flow.report()

The output specifies that we've actually run two calculations here:  a `WFN` calculation where we calculate wavefunctions using Quantum ESPRESSO, and `PW2BGW` where we convert the resulting Quantum-ESPRESSO-specific output files into a format readable by BerkeleyGW.  Unlike in the density case where we ran a single task, here we're running two tasks (`WFN` and `PW2BGW`) in a workflow (hence the name `QeBgwFlow`).

# WFNq #

Next, we'll create `WFNq`, which is the "standard" k-shifted and q-shifted wavefunction file which is read by the `Epsilon` calculation, and thus is needed for all BerkeleyGW calculations.

The only dictionary we need to create is are the settings specific to the `WFNq` wavefunction:

In [ ]:
wfnq_settings = {
    'ecutwfc': qe_wfn_settings['ecutwfc'],
    'ngkpt': qe_wfn_settings['ngkpt'],
    'kshift': qe_wfn_settings['kshift'],
    'qshift': [.001, .0, .0],
}

And then we can prepare the calculation:

In [ ]:
wfnq_flow = QeBgwFlow(
    dirname='Runs/13-Wfnq',
    structure=structure,
    **qe_general_settings,
    **qe_wfn_input_files,
    **wfnq_settings,
    **mpi_settings,
)

Create and run it:

In [ ]:
wfnq_flow.write()
wfnq_flow.run()
wfnq_flow.report()

## Wfn_co ##

Next, we'll create `WFN_co`, which is the wavefunction on a coarser (and unshifted) grid than `WFN`.  This is used by `Sigma`, `Kernel`, and `Absorption`, and thus will be needed by most BerkeleyGW calculations.  we will also use this calculation to generate the ground state density and exchange-correlation energy density that will be used by `Sigma`.

Once again, we set up the dictionary with our needed variables:

In [ ]:
wfn_co_settings = {
    'ecutwfc': qe_wfn_settings['ecutwfc'],
    'ngkpt': [2, 2, 2],
    'kshift': [.0, .0, .0],
    'nbnd': 9,
    'rhog_flag': True,
}

Note that there's a new flag `rhog_flag` which tells `QeBgwFlow` to generate additional density-related files.

Now we can prepare the calculation:

In [ ]:
mpi_settings['nproc'] = 1
wfn_co_flow = QeBgwFlow(
    dirname = 'Runs/14-Wfn_co',
    structure = structure,
    **qe_general_settings,
    **qe_wfn_input_files,
    **wfn_co_settings,
    **mpi_settings,
)

And create and run it:

In [ ]:
wfn_co_flow.write()
wfn_co_flow.run()
wfn_co_flow.report()

## WFN_fi ##

Next, we'll create `WFN_fi`, the k-shifted `WFN` on a finer grid than `WFN`.  This is used during interpolation in the `Absorption` executable and thus is only needed if you need to solve the BSE equations.  (Symmetry is also turned off for this calculation.)

By this point, you're probably familiar with the steps, so we'll present them without comment:

In [ ]:
wfn_fi_settings = {
    'ecutwfc': qe_wfn_settings['ecutwfc'],
    'ngkpt': [2, 2, 2],
    'kshift': [.5, .5, .5],
    'nbnd': 9,
    'symkpt': False,
}

In [ ]:
wfn_fi_flow = QeBgwFlow(
    dirname = 'Runs/15-Wfn_fi',
    structure = structure,
    **qe_general_settings,
    **qe_wfn_input_files,
    **wfn_fi_settings,
    **mpi_settings,
)

In [ ]:
wfn_fi_flow.write()
wfn_fi_flow.run()
wfn_fi_flow.report()

## WFNq_fi ##

FINALLY, we'll create `WFNq_fi`, the k-shifted and q-shifted `WFN` on a finer grid than `WFN`.  Like `WFN_fi`, this is used during interpolation in the `Absorption` executable and thus is only needed if you need to solve the BSE equations.  (And symmetry is turned off, as before.)

Let's go through the steps again:

In [ ]:
wfnq_fi_settings = {
    'ecutwfc': qe_wfn_settings['ecutwfc'],
    'ngkpt': qe_wfn_fi_settings['ngkpt'],
    'kshift': qe_wfn_fi_settings['kshift'],
    'nbnd': qe_wfn_fi_settings['nbnd'],
    'symkpt': qe_wfn_fi_settings['symkpt'],
    'qshift': [.001, .0, .0],
}

In [ ]:
wfnq_fi_flow = QeBgwFlow(
    dirname = 'Runs/16-Wfnq_fi',
    structure = structure,
    **qe_general_settings,
    **qe_wfn_input_files,
    **wfnq_fi_settings,
    **mpi_settings,
)                                                                            

In [ ]:
wfnq_fi_flow.write()
wfnq_fi_flow.run()
wfnq_fi_flow.report()

# Running GW #

Now the moment you've been waiting for, when we actually run a GW calculation!

## Epsilon ##

Our first step is to run an `Epsilon` calculation, where we'll generate the dielectric matrix (to be precise, the inverse of the dielectric matrix.)

Because BerkeleyGW uses a file-based communication system, we'll need to specify the location of the wavefunction files that we previously calculated:

In [ ]:
epsilon_input_files = dict(
    wfn_fname='Runs/12-Wfn/wfn.cplx',
    wfnq_fname='Runs/13-Wfnq/wfn.cplx',
)

Actually, we can set the file name above using a property of the flow

In [ ]:
epsilon_input_files = dict(
    wfn_fname=wfn_flow.wfn_fname,
    wfnq_fname=wfnq_flow.wfn_fname,
)

As well as the settings for an `Epsilon` calculation:

In [ ]:
epsilon_settings = dict(
    ngkpt = wfn_settings['ngkpt'],    #    'ngkpt': [2, 2, 2],
    qshift = wfnq_settings['qshift'], #    'qshift': [.001, .0, .0],
    ecuteps = 10.0,
)

And then we can prepare the Epsilon calculation using an `EpsilonTask` object (reusing our `mpi_settings` dictionary from before):

In [ ]:
epsilon_task = EpsilonTask(
    dirname='Runs/21-Epsilon',
    structure=structure,
    **epsilon_input_files,
    **epsilon_settings,
    **mpi_settings)

Let's run the calculation:

In [ ]:
epsilon_task.write()
epsilon_task.run()
epsilon_task.report()

## Sigma ##

Now that we've calculated the (inverse) dielectric matrix and needed wavefunctions, we have everything we need to calculate the GW self-energy.  This is done with the `Sigma` executable, which takes as inputs the results from our `WFN_co` and `Epsilon` calculations:

In [ ]:
sigma_input_files = dict(
    wfn_co_fname='Runs/14-Wfn_co/wfn.cplx',
    rho_fname='Runs/14-Wfn_co/rho.cplx',
    vxc_dat_fname='Runs/14-Wfn_co/vxc.dat',
    
    eps0mat_fname='Runs/21-Epsilon/eps0mat.h5',
    epsmat_fname='Runs/21-Epsilon/epsmat.h5',
)

Then again, making use of the object properties, we can get the above file names with

In [ ]:
sigma_input_files = dict(
    wfn_co_fname=wfn_co_flow.wfn_fname,
    rho_fname=wfn_co_flow.rho_fname,
    vxc_dat_fname=wfn_co_flow.vxc_dat_fname,
    
    eps0mat_fname=epsilon_task.eps0mat_fname,
    epsmat_fname=epsilon_task.epsmat_fname,
)

Specify the settings:

In [ ]:
sigma_settings = dict(
    ngkpt = wfn_co_settings['ngkpt'],  # ngkpt': [2,2,2],
    ibnd_min = 1,           # Minimum band for GW corrections
    ibnd_max = 8,           # Maximum band for GW corrections
    #'extra_lines' : ['dont_use_hdf5'],
)

Prepare the calculation:

In [ ]:
sigma_task = SigmaTask(
    dirname='Runs/22-Sigma',
    structure=structure,
    **sigma_input_files,
    **sigma_settings,
    **mpi_settings)

And finally run it.

In [ ]:
# Execution
sigma_task.write()
sigma_task.run()
sigma_task.report()

If you see an `Unfinised` status, something went wrong, and you should inspect the content of the run directory, in particular the main output file `Runs/22-Sigma/sigma.out` .
Make sure you are using the latest version of BerkeleyGW.

If you see a `Completed` status, then congratulations! You have successfully ran a BerkeleyGW calculation from start to finish.

# Running BSE #

For those of you that want to go further, BerkeleyGW can calculate excitionic properties on the GW+BSE level of theory.  This is done with the `KernelTask` and `AbsorptionTask` classes.

## Kernel ##

`Kernel` takes in as inputs the results of `WFN_co` and `Epsilon`:

In [ ]:
kernel_input_files = dict(
    wfn_co_fname=wfn_co_flow.wfn_fname,
    eps0mat_fname=epsilon_task.eps0mat_fname,
    epsmat_fname=epsilon_task.epsmat_fname,
)

We can specify its settings:

In [ ]:
kernel_settings = dict(
    ngkpt = wfn_co_settings['ngkpt'],
    ecuteps = epsilon_settings['ecuteps'],
    nbnd_val = 4,
    nbnd_cond = 4,
    # These extra lines will be added verbatim to the input file.
    extra_lines = ['use_symmetries_coarse_grid', 'screening_semiconductor'],
)

Prepare the calculation:

In [ ]:
kernel_task = KernelTask(
    dirname='Runs/23-Kernel',
    structure=structure,
    **kernel_input_files,
    **kernel_settings,
    **mpi_settings)

And finally run it:

In [ ]:
kernel_task.write()
kernel_task.run()
kernel_task.report()

## Absorption ##

Finally, we solve the BSE equation via the `Absorption` executable.  It has as inputs the results of `WFN_co`, `WFNq_fi`, and `WFN_fi`, as well as all previous BerkleyGW executables `Epsilon`, `Sigma`, and `Kernel`:

In [ ]:
absorption_input_files = dict(
    wfn_co_fname = 'Runs/14-Wfn_co/wfn.cplx',
    wfn_fi_fname = 'Runs/15-Wfn_fi/wfn.cplx',
    wfnq_fi_fname = 'Runs/16-Wfnq_fi/wfn.cplx',
    
    eps0mat_fname = 'Runs/21-Epsilon/eps0mat.h5',
    epsmat_fname = 'Runs/21-Epsilon/epsmat.h5',
    eqp_fname = 'Runs/22-Sigma/eqp1.dat',
    bsemat_fname = 'Runs/23-Kernel/bsemat.h5'

    # If you don't use hdf5, the BSE matrix is written in two separate files.
    #bsexmat_fname = 'Runs/23-Kernel/bsexmat',
    #bsedmat_fname = 'Runs/23-Kernel/bsedmat',
)

Or, using the appropriate variables,

In [ ]:
absorption_input_files = dict(
    wfn_co_fname = wfn_co_flow.wfn_fname,
    wfn_fi_fname = wfn_fi_flow.wfn_fname,
    wfnq_fi_fname = wfnq_fi_flow.wfn_fname,
    
    eps0mat_fname = epsilon_task.eps0mat_fname,
    epsmat_fname = epsilon_task.epsmat_fname,
    eqp_fname = sigma_task.eqp1_fname,
    bsemat_fname = kernel_task.bsemat_fname,

    # If you don't use hdf5, the BSE matrix is written in two separate files.
    #bsexmat_fname = kernel_task.bsexmat_fname,
    #bsedmat_fname = kernel_task.bsedmat_fname,
)

Next, we set the calculation settings. There are...a lot of those.

In [ ]:
absorption_settings = dict(
    ngkpt = [2, 2, 2],        # k-points grid
    nbnd_val = 4,             # Number of valence bands
    nbnd_cond = 4,            # Number of conduction bands
    nbnd_val_co = 4,          # Number of valence bands on the coarse grid
    nbnd_cond_co = 4,         # Number of conduction bands on the coarse grid
    nbnd_val_fi = 4,          # Number of valence bands on the fine grid
    nbnd_cond_fi = 4,         # Number of conduction bands on the fine grid

    # These extra lines will be added verbatim to the input file.
    extra_lines = [
        'use_symmetries_coarse_grid',
        'no_symmetries_fine_grid',
        'no_symmetries_shifted_grid',
        'screening_semiconductor',
        'use_velocity',
        'gaussian_broadening',
        'eqp_co_corrections',
    ],
    # These extra variables will be added to the input file as '{variable} {value}'.
    extra_variables = {
        'energy_resolution': 0.15,
    },
)

But preparing the calculation is as simple as always:

In [ ]:
absorption_task = AbsorptionTask(
    dirname='Runs/24-Absorption',
    structure=structure,
    **absorption_input_files,
    **absorption_settings,
    **mpi_settings)

And, at last, we can run it.

In [ ]:
absorption_task.write()
absorption_task.run()
absorption_task.report()

Congratulations yet again!  You've run a full GW+BSE calculation!

# Using workflows #

Can we do all of these steps at once? Yes we can!

In [ ]:
from BGWpy import GWFlow, BSEFlow

In [ ]:
flow = GWFlow(
    dirname='Runs/32-GW',
    dft_flavor='espresso',

    structure = Structure.from_file('../Data/Structures/GaAs.json'),
    prefix = 'GaAs',
    pseudo_dir = '../Data/Pseudos',
    pseudos = ['31-Ga.PBE.UPF', '33-As.PBE.UPF'],

    ecutwfc = 10.0,
    nbnd = 9,

    ngkpt = [2,2,2],
    kshift = [.5,.5,.5],
    qshift = [.001,.0,.0],
    ibnd_min = 1,
    ibnd_max = 8,
    ecuteps = 10.0,

    # Extra lines and extra variables
    epsilon_extra_lines = [],
    epsilon_extra_variables = {},
    sigma_extra_lines = ['screening_semiconductor'],
    sigma_extra_variables = {},

    **mpi_settings)

Let's execute the whole thing.

In [ ]:
flow.write()
flow.run()
flow.report()

Likewise, for the BSE

In [ ]:
flow = BSEFlow(
    dirname='Runs/33-BSE',
    dft_flavor='espresso',

    structure = Structure.from_file('../Data/Structures/GaAs.json'),
    prefix = 'GaAs',
    pseudo_dir = '../Data/Pseudos',
    pseudos = ['31-Ga.PBE.UPF', '33-As.PBE.UPF'],

    ecutwfc = 10.0,
    nbnd = 12,
    nbnd_fine = 9,

    ngkpt = [2,2,2],
    kshift = [.5,.5,.5],
    qshift = [.001,.0,.0],

    # Fine grids
    ngkpt_fine = [4,4,4],
    kshift_fine = [.0,.0,.0],

    ibnd_min = 1,
    ibnd_max = 8,
    ecuteps = 10.0,
    sigma_extra_lines = ['screening_semiconductor'],

    # Kernel variables
    nbnd_val = 4,
    nbnd_cond = 4,

    kernel_extra_lines = [
        'use_symmetries_coarse_grid',
        'screening_semiconductor',
        ],

    # Absorption variables
    nbnd_val_co=4,
    nbnd_cond_co=4,
    nbnd_val_fi=4,
    nbnd_cond_fi=4,
    
    absorption_extra_lines = [
        'use_symmetries_coarse_grid',
        'no_symmetries_fine_grid',
        'no_symmetries_shifted_grid',
        'screening_semiconductor',
        'use_velocity',
        'gaussian_broadening',
        'eqp_co_corrections',
        ],
    
    absorption_extra_variables = {
        'energy_resolution' : 0.15,
        },
    
    **mpi_settings)

In [ ]:
flow.write()
flow.run()
flow.report()

## Custom workflows ##

For a realistic GW or BSE calculation, in general, you don't run every steps all at once like we did. You actually perform a **convergence study**, in which you gradually increase the parameters until the calculation is converged. For example, in a GW calculation, we have the following convergence studies to perform:

* Convergence of the k-points grids for epsilon
* Convergence of the q-points grid for sigma
* Convergence on the number of bands for epsilon
* Convergence on the number of bands for sigma
* Convergence on the size of the dielectric matrix

For these, you will need to construct your own workflow. Here is an example.

In [ ]:
from os.path import join as pjoin
from BGWpy import Workflow

workflow = Workflow(dirname='Runs/50-Workflow')


epsilon_input_files = dict(
    wfn_fname=wfn_flow.wfn_fname,
    wfnq_fname=wfnq_flow.wfn_fname,
)

sigma_input_files = dict(
    wfn_co_fname=wfn_co_flow.wfn_fname,
    rho_fname=wfn_co_flow.rho_fname,
    vxc_dat_fname=wfn_co_flow.vxc_dat_fname,

)

ecuteps_l = [5.0, 7.5, 10.0]

for i, ecuteps in enumerate(ecuteps_l):

    epsilon_settings['ecuteps'] = ecuteps
    
    epsilon_task = EpsilonTask(
        dirname=pjoin(workflow.dirname, 'Epsilon{}'.format(i)),
        structure=structure,
        **epsilon_input_files,
        **epsilon_settings,
        **mpi_settings)
    
    sigma_task = SigmaTask(
        dirname=pjoin(workflow.dirname, 'Sigma{}'.format(i)),
        structure=structure,

        eps0mat_fname=epsilon_task.eps0mat_fname,
        epsmat_fname=epsilon_task.epsmat_fname,
        
        **sigma_input_files,
        **sigma_settings,
        **mpi_settings)

    workflow.add_tasks([epsilon_task, sigma_task])

In [ ]:
workflow.write()
workflow.run()
workflow.report()

Note that you could also run and report each task sequentially with

In [ ]:
for task in workflow.tasks:
    task.run()
    task.report()

And of course, you should now check the result of the calculations in the different output files, and plot the convergence of the quasiparticle energies as a function of ecuteps.

# Closing word #

`BGWpy` allows you to use pre-defined workflows and custom ones. However, it is your own responsibility to check every input parameter, and verify the convergence of the results. Happy computing!